In [1]:
import pandas as pd
import numpy as np

m15 = []
m30 = []
h1 = []
h4 = []
d1 = []
hours = [
    "00:00",
    "01:00",
    "02:00",
    "03:00",
    "05:00",
    "06:00",
    "07:00",
    "09:00",
    "10:00",
    "11:00",
    "13:00",
    "14:00",
    "15:00",
    "17:00",
    "18:00",
    "19:00",
    "21:00",
    "22:00",
    "23:00",
]
four_hours = ["04:00", "08:00", "12:00", "16:00", "20:00"]
timeframes = [m15, m30, h1, h4, d1]
timeframe_names = ['m15', 'm30', 'h1', 'h4', 'd1']
symbols = [
    "GBPUSD_2021_all.csv",
    "GBPUSD_2020_all.csv",
    "GBPUSD_2019_all.csv",
    "GBPJPY_2021_all.csv",
    "GBPJPY_2020_all.csv",
    "GBPJPY_2019_all.csv",
    "EURUSD_2021_all.csv",
    "EURUSD_2020_all.csv",
]

def processSymbol(symbol):
    symbol_path = '/media/bruno/f71fc4fd-abcd-4a4b-9fb1-6a8899041c1c/FX-Training-Data/ALL/' + str(symbol)
    base_data = np.array(
    pd.read_csv(
        symbol_path,
        header=None,
    )
)
    return base_data


In [2]:

def makeTimeframeData(base_data):
    print('making timeframe data')
    for i in range(len(base_data)):
        if str(base_data[i][1]).endswith("00"):
            if str(base_data[i][1]) == "00:00":
                d1.append(i)
                h4.append(i)
                h1.append(i)
                m30.append(i)
                m15.append(i)
                continue
            elif str(base_data[i][1]) in hours:
                h1.append(i)
                m30.append(i)
                m15.append(i)
                continue
            elif str(base_data[i][1]) in four_hours:
                h1.append(i)
                h4.append(i)
                m30.append(i)
                continue
        elif str(base_data[i][1]).endswith("15"):
            m15.append(i)
            continue
        elif str(base_data[i][1]).endswith("30"):
            m30.append(i)
            m15.append(i)
            continue
        elif str(base_data[i][1]).endswith("45"):
            m15.append(i)
            continue


In [3]:
class Row:
    def __init__(self, date, time, open_p, high, low, close, volume, state):
        self.date = date
        self.time = time
        self.open_p = float(open_p)
        self.high = float(high)
        self.low = float(low)
        self.close = float(close)
        self.volume = int(volume)
        self.state = state

    def makeRecord(self):
        return list(
            [
                self.date,
                self.time,
                self.open_p,
                self.high,
                self.low,
                self.close,
                self.volume,
                self.state,
            ]
        )


In [4]:
def getMax(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return max(values)

def getMin(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return min(values)


def getSum(base_data, column_index, start, stop):
    values = list()
    for i in range(start, stop):
        values.append(float(base_data[i][column_index]))
    return sum(values)



So now that you have lists of positions for each time interval, use those to recalculate OHLCVS(indices 2,3,4,5,6,7)
<!-- for each index listed in h1,h4,etc look for a corresponding element in base_data -->
<!-- open = same open as that index -->
<!-- close = close of index prior to interval index -->
<!-- high = highest value base_data[index][3] in range(curr_interval_index, next_interval_index) -->
<!-- low =  lowest value base_data[index][3] in range(curr_interval_index, next_interval_index)-->
<!-- volume = sum of base_data[index][6] in range(curr_interval_index, next_interval_index) -->
<!-- state = [0 if open==close, 1 if close>open, -1 if close<open] -->

In [5]:
def processCsv(base_data, timeframe, symbol):
    data = [["date", "time", "open", "high", "low", "close", "volume", "state"]]

    for i in range(len(timeframe)):
        # if i == 0:
        #     # data = [["date", "time", "open", "high", "low", "close", "volume", "state"]]
        #     print("starting afresh")
        if (i + 1) == len(timeframe):
            data = pd.DataFrame(data)
            newCsv_name = (
                symbol[:11]
                + "_"
                + str(timeframe_names[timeframes.index(timeframe)])
                + ".csv"
            )
            data.to_csv(newCsv_name, header=False)
            break

        row = Row(
            date=base_data[timeframe[i]][0],
            time=base_data[timeframe[i]][1],
            open_p=base_data[timeframe[i]][2],
            high=getMax(base_data, 3, timeframe[i], timeframe[i + 1]),
            low=getMin(base_data, 4, timeframe[i], timeframe[i + 1]),
            close=base_data[(timeframe[i + 1]) - 1][5],
            volume=getSum(base_data, 6, timeframe[i], timeframe[i + 1]),
            state=0,
        )
        if row.close > row.open_p:
            row.state = 1
        elif row.close < row.open_p:
            row.state = -1
        else:
            row.state = 0
        data.append(row.makeRecord())


In [6]:
def clrLists(timeframes):
    for tf in timeframes:
        tf.clear()

In [7]:



for sy in symbols:
    clrLists(timeframes=timeframes)
    base_data = processSymbol(sy)
    makeTimeframeData(base_data= base_data)
    for tf in timeframes:
        print('length of base_data :{}, symbol:{}'.format(str(len(base_data)), str(sy)))
        print('length of timeframe_data : {}'.format(str(len(tf))))
        processCsv(base_data= base_data, timeframe=tf,symbol=sy)
        
        


making timeframe data
length of base_data :372321, symbol:GBPUSD_2021_all.csv
length of timeframe_data : 23603
starting afresh
length of base_data :372321, symbol:GBPUSD_2021_all.csv
length of timeframe_data : 12472
starting afresh
length of base_data :372321, symbol:GBPUSD_2021_all.csv
length of timeframe_data : 6244
starting afresh
length of base_data :372321, symbol:GBPUSD_2021_all.csv
length of timeframe_data : 1564
starting afresh
length of base_data :372321, symbol:GBPUSD_2021_all.csv
length of timeframe_data : 264
starting afresh
making timeframe data
length of base_data :372294, symbol:GBPUSD_2020_all.csv
length of timeframe_data : 23565
starting afresh
length of base_data :372294, symbol:GBPUSD_2020_all.csv
length of timeframe_data : 12433
starting afresh
length of base_data :372294, symbol:GBPUSD_2020_all.csv
length of timeframe_data : 6207
starting afresh
length of base_data :372294, symbol:GBPUSD_2020_all.csv
length of timeframe_data : 1528
starting afresh
length of base_da

/tmp/ipykernel_5117/2910683434.py:47: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(


making timeframe data
length of base_data :373568, symbol:GBPJPY_2020_all.csv
length of timeframe_data : 23616
starting afresh
length of base_data :373568, symbol:GBPJPY_2020_all.csv
length of timeframe_data : 12452
starting afresh
length of base_data :373568, symbol:GBPJPY_2020_all.csv
length of timeframe_data : 6216
starting afresh
length of base_data :373568, symbol:GBPJPY_2020_all.csv
length of timeframe_data : 1538
starting afresh
length of base_data :373568, symbol:GBPJPY_2020_all.csv
length of timeframe_data : 239
starting afresh
making timeframe data
length of base_data :372558, symbol:GBPJPY_2019_all.csv
length of timeframe_data : 23554
starting afresh
length of base_data :372558, symbol:GBPJPY_2019_all.csv
length of timeframe_data : 12428
starting afresh
length of base_data :372558, symbol:GBPJPY_2019_all.csv
length of timeframe_data : 6213
starting afresh
length of base_data :372558, symbol:GBPJPY_2019_all.csv
length of timeframe_data : 1551
starting afresh
length of base_da